In [2]:
from langchain_core.prompts import ChatPromptTemplate

In [6]:
from langchain_community.llms import Ollama

In [7]:
llm = Ollama(model="llama2")

In [9]:
prompt = ChatPromptTemplate.from_template("Tell me postive things for {input}")
chain = prompt | llm

In [10]:
chain.invoke({"input": "For today morning"})

"\nOf course! Here are some positive things to start your day off right:\n\n1. The sun is shining brightly, bringing new life and energy to the world.\n2. You woke up feeling refreshed and rested after a good night's sleep.\n3. You have a delicious breakfast waiting for you, ready to fuel your body for the day ahead.\n4. Today is a new day, full of opportunities and possibilities.\n5. You have a plan in place to tackle your tasks and responsibilities, and you're feeling motivated to make progress.\n6. You have supportive friends and family who care about you and want to help you succeed.\n7. You have the power to choose how you want to spend your time and energy today, and you get to decide what brings you joy and fulfillment.\n8. The world is full of beauty and wonder, just waiting to be discovered and appreciated.\n9. You are capable of learning and growing every day, and you're excited to see how far you can go.\n10. Today is a gift, and you get to unwrap it and enjoy every moment.\

In [11]:
from langchain_core.output_parsers import StrOutputParser

In [12]:
output_parser = StrOutputParser()

In [13]:
chain = prompt | llm | output_parser

In [14]:
chain.invoke({"input": "what is langsmith and how can it help?"})

"\nLangsmuir, the inventor of the Langsmuir-Hinshelwood model of chemical reactions, was a pioneer in the field of surface chemistry. His work laid the foundation for our understanding of catalysis, surface reaction kinetics, and the behavior of molecules at interfaces. Here are some positive aspects of Langsmith's contributions:\n\n1. Advances in Catalysis: Langsmith's model of chemical reactions has been instrumental in developing a deeper understanding of catalytic processes. By studying the adsorption and desorption of reactants on surfaces, he was able to explain the mechanisms behind many important catalytic reactions.\n2. Surface Reaction Kinetics: Langsmith's work on surface reaction kinetics has provided valuable insights into the rates of chemical reactions at interfaces. His models have helped to elucidate the factors that influence the rate of reactions, such as temperature, pressure, and surface properties.\n3. Molecular Behavior at Interfaces: Langsmith's research on mole

In [15]:
from langchain_community.embeddings import OllamaEmbeddings

In [16]:
embeddings = OllamaEmbeddings()

In [17]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()


from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [20]:
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>

                                          Question: {input}


""")

In [21]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [22]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Who is Sachin Tendulkar?",
    "context": [Document(page_content="Sachin Tendulkar is the greatest cricketer")]
})

'Of course! Based on the provided context, Sachin Tendulkar is a renowned Indian cricketer widely regarded as one of the greatest batsmen in the history of cricket.'

In [24]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
response = retrieval_chain.invoke({"input": "How can langsmith help with testing?"})
print(response["answer"])

 Based on the provided context, LangSmith can help with testing in several ways:

1. Prototyping: LangSmith provides a platform for rapid iteration and experimentation, allowing developers to test out different prompts and models quickly.
2. Debugging: When developing new LLM applications, LangSmith tracing is enabled by default, providing clear visibility and debugging information at each step of an LLM sequence. This helps identify and root-cause issues when things go wrong.
3. Initial Test Set: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
4. Comparison View: When prototyping different versions of an application and making changes, LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of the 

In [27]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [32]:
prompt = ChatPromptTemplate.from_messages([
MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query that will take a look at the previous conversation for the answers")

    
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [33]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLM application?")]

retriever_chain.invoke({

    "chat_histiry": chat_history,
    "input": "Tell me now"
})

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
 Document(page_content="This allows you to quickly test out different prompts and models. You can open the playground from any prompt or model run in your trace.\nEvery playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retrieval_chain, document_chain)

In [39]:
#need to fix this

chat_history = [HumanMessage(content="Can langsmith help test my llm application"), AIMessage(content="Yes! definitelly!!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how",
})

AttributeError: 'str' object has no attribute 'page_content'